In [1]:
from astropy.io import fits
import numpy as np
from astropy.table import QTable,vstack
import astropy.units as u
import astropy.utils
from astropy.coordinates import SkyCoord
import matplotlib.pyplot as plt
import matplotlib.colors
from scipy.stats import chisquare
from scipy.stats import chi2
from scipy.stats import lognorm
from scipy.stats import f
from astropy.timeseries import LombScargle
import time
from astropy.visualization import hist
from scipy.optimize import curve_fit
from astropy.timeseries import BoxLeastSquares
from astropy.timeseries import TimeSeries
from astropy.time import Time
from tqdm import tqdm


import functions
#import upsilon

In [ ]:
t_zr=QTable.read('t_zr_1024.ecsv')
t_zg=QTable.read('t_zg_1024.ecsv')

In [13]:
t=np.unique(t_zr['SourceID'])
ra,dec=np.zeros(len(t)),np.zeros(len(t))
for i,row in enumerate(t):
    ra[i]=np.mean(t_zr[t_zr['SourceID']==row]['RA'])
    dec[i]=np.mean(t_zr[t_zr['SourceID']==row]['DEC'])

In [18]:
zr=astropy.table.Table([t,ra,dec],names=['SourceID','RA','DEC'])

In [20]:
t=np.unique(t_zg['SourceID'])
ra,dec=np.zeros(len(t)),np.zeros(len(t))
for i,row in enumerate(t):
    ra[i]=np.mean(t_zg[t_zg['SourceID']==row]['RA'])
    dec[i]=np.mean(t_zg[t_zg['SourceID']==row]['DEC'])

In [21]:
zg=astropy.table.Table([t,ra,dec],names=['SourceID','RA','DEC'])

In [23]:
zr.write('simbad_zr.csv')
zg.write('simbad_zg.csv')

In [28]:
simbad_zr=QTable.read('zr_simbad.csv')
simbad_zg=QTable.read('zg_simbad.csv')

In [74]:
t_zr=QTable.read('t_zr_1024.ecsv')
t_zg=QTable.read('t_zg_1024.ecsv')

In [75]:
t_zr['identifier']=np.empty(1,dtype='<U256')
t_zr['dist']=np.nan
t_zr['type']=np.empty(1,dtype='<U30')

In [76]:
t_zg['identifier']=np.empty(1,dtype='<U256')
t_zg['dist']=np.nan
t_zg['type']=np.empty(1,dtype='<U30')

In [77]:
for i,row in enumerate(t_zr):
    common=simbad_zr[simbad_zr['SourceID']==row['SourceID']]
    if len(common)!=0:
        match=common[common['angDist'].argmin()]
        row['identifier']=match['main_id']
        row['dist']=match['angDist']
        row['type']=match['main_type']

In [79]:
for i,row in enumerate(t_zg):
    common=simbad_zg[simbad_zg['SourceID']==row['SourceID']]
    if len(common)!=0:
        match=common[common['angDist'].argmin()]
        row['identifier']=match['main_id']
        row['dist']=match['angDist']
        row['type']=match['main_type']

In [82]:
t_zr.write('t_zr_1212.ecsv')
t_zg.write('t_zg_1212.ecsv')

In [2]:
t_zr=QTable.read('t_zr_1212.ecsv')
t_zg=QTable.read('t_zg_1212.ecsv')
t_zr=t_zr[np.isnan(t_zr['LNP_SIG'])==False]
t_zg=t_zg[np.isnan(t_zg['LNP_SIG'])==False]

var_zr=t_zr[t_zr['sf']<1/(100*len(t_zr))] #99% chance variabe
var_zg=t_zg[t_zg['sf']<1/(100*len(t_zg))] #99% chance variabe

nonvar_zr=t_zr[t_zr['sf']>1/(100*len(t_zr))]
nonvar_zg=t_zg[t_zg['sf']>1/(100*len(t_zg))]

def cut_LNP_SIG(t,period,LNP_SIG):
    #t=t[(t['period']<period[0])|(t['period']>period[1])]
    #t=t[(t['period']<period[2])|(t['period']>period[3])]
    #if len(period)>4:
        #t=t[(t['period']<period[4])|(t['period']>period[5])]
    t=t[t['LNP_SIG']<LNP_SIG]
    return t

t_zr=cut_LNP_SIG(var_zr,_,1/(100*len(t_zr)))
t_zg=cut_LNP_SIG(var_zg,_,1/(100*len(t_zg)))


# cut power ratios
obj_zr=t_zr[t_zr['power']/t_zr['power_c']>=1]
obj_zg=t_zg[t_zg['power']/t_zg['power_c']>=1]


# cut close periods
obj_zr=obj_zr[np.abs(obj_zr['period']-obj_zr['period_c'])/obj_zr['period_c']>1e-2]
obj_zg=obj_zg[np.abs(obj_zg['period']-obj_zg['period_c'])/obj_zg['period_c']>1e-2]

obj_zr=obj_zr[(np.abs(obj_zr['period']-obj_zr['period_c'])/obj_zr['period_c']<0.49)|(np.abs(obj_zr['period']-obj_zr['period_c'])/obj_zr['period_c']>0.91)|(obj_zr['LNP_SIG']<1e-30)]
obj_zg=obj_zg[(np.abs(obj_zg['period']-obj_zg['period_c'])/obj_zg['period_c']<0.49)|(np.abs(obj_zg['period']-obj_zg['period_c'])/obj_zg['period_c']>0.91)|(obj_zg['LNP_SIG']<1e-30)]




# restore original table for ploting reasons
t_zr=QTable.read('t_zr_1212.ecsv')
t_zg=QTable.read('t_zg_1212.ecsv')

# choose only OID with highest numobs
mask=np.zeros(len(obj_zr))+1
for i,obj in enumerate(obj_zr):
    copy=t_zr[t_zr['SourceID']==obj['SourceID']]
    if len(copy)==1:
        if obj['numobs']<50:
            mask[i]=0
    if len(copy)>1:
        if obj['OID']!=copy['OID'][copy['numobs'].argmax()]:
            mask[i]=0
        if obj['numobs']<50:
            mask[i]=0
obj_zr=obj_zr[mask==1]


mask=np.zeros(len(obj_zg))+1
for i,obj in enumerate(obj_zg):
    copy=t_zg[t_zg['SourceID']==obj['SourceID']]
    if len(copy)==1:
        if obj['numobs']<50:
            mask[i]=0
    if len(copy)>1:
        if obj['OID']!=copy['OID'][copy['numobs'].argmax()]:
            mask[i]=0
        if obj['numobs']<50:
            mask[i]=0
obj_zg=obj_zg[mask==1]

In [3]:
unobj_zr=var_zr[np.isin(var_zr['SourceID'],obj_zr['SourceID'])==0]
unobj_zg=var_zg[np.isin(var_zg['SourceID'],obj_zg['SourceID'])==0]

In [4]:
len(np.unique(obj_zr[np.isnan(obj_zr['dist'])==0]['SourceID'])),len(np.unique(obj_zg[np.isnan(obj_zg['dist'])==0]['SourceID']))

(14, 3)

In [5]:
len(np.unique(unobj_zr[np.isnan(unobj_zr['dist'])==0]['SourceID'])),len(np.unique(unobj_zg[np.isnan(unobj_zg['dist'])==0]['SourceID']))

(43, 28)

In [6]:
obj_zr[np.isnan(obj_zr['dist'])==0].show_in_notebook()

idx,SourceID,filter,OID,RA,DEC,numobs,DELTA_SIG_MAG,CONST_CHI2,sf,rmse,LNP_SIG,period,period_c,power,power_c,mean,std,min,max,sigma_mean,sigma_std,gaussian1_err,gaussian2_err,gumbel_err,g2_flag,Pg1g2,Pgug2,Pg1gu,shape,upsilon_class,upsilon_prob,upsilon_flag,upsilon_period,identifier,dist,type
0,52,zr,806203100023276,5.0806496,59.3093092,664,48.39557647705078,9.548270702053268,1.3227187319910005e-189,0.3108588755130768,1.7018337280194596e-78,595.2450708466688,0.997336033066433,0.6229683753591274,0.21949631231274336,19.675853729248047,0.3108588755130768,19.17412567138672,20.918895721435547,-0.7161276936531067,3.285275936126709,0.05060821258505381,0.007833020026423389,0.026125014733712922,0.0,0.014227170500624942,0.0517871312497491,0.20132354399945207,g2,LPV_OSARG_AGB,0.33,0.0,566.9630815875007,[HL90] 61,1.854526,HII
1,77,zr,806203100063307,5.0870906,59.2998095,836,344.7351989746094,40.842491010031104,0.0,0.22717958688735962,7.488112388238467e-24,26.55255504914417,0.997336033066433,0.25554257162342087,0.0034232661997002983,18.791976928710938,0.22717958688735962,17.675485610961914,19.1998348236084,-0.8078729510307312,6.599571704864502,0.010941253454579771,0.0022451831278411214,0.04598158685382897,1.0,0.024749755228021697,0.001446042828369909,0.961198797591372,g1,CEPH_Other,0.44,0.0,26.524588612280738,[WWW2005] XA 11,0.264632,X
2,90,zr,806203100056957,5.0836327,59.3114535,846,567.4482421875,120.79296419281002,0.0,0.3739984333515167,3.247011689208663e-280,2043.207284960579,0.997336033066433,0.9506730073393318,0.47493011002121566,18.124500274658203,0.3739984333515167,17.195171356201172,18.949966430664062,-2.5609970092773438,11.737095832824707,0.18947997991941207,0.020160519579386485,0.15060952910358819,0.0,0.006788985925998747,0.01068657840378251,0.38484022277800156,g2,LPV_Mira_AGB_C,0.26,0.0,2267.852326350003,LGGS J002020.03+591841.2,0.205743,Em*
3,145,zr,806203100043772,5.0957151,59.2994528,717,66.61664581298828,3.5379984089296954,2.2753838245357322e-18,0.14171883463859558,1.0515294240953812e-12,981.7766691905116,0.997336033066433,0.17591322363193937,0.09890003454859338,19.4619083404541,0.14171883463859558,18.99640655517578,20.149547576904297,-0.18844391405582428,1.9061943292617798,0.014671911593230497,0.003428865224048767,0.0160385433525674,0.0,0.03191100590870233,0.026816135608569547,0.5452531399277134,g2,NonVar,0.35,0.0,1133.9261631750014,LGGS J002022.99+591757.8,0.329861,Em*
4,158,zr,806203100057075,5.0953672,59.297784,702,85.88482666015625,6.528562040377028,1.9791103413711206e-94,0.179270938038826,5.255097789862924e-52,595.2450708466688,0.997336033066433,0.4806277327856466,0.1408160323876546,19.39754295349121,0.179270938038826,18.829944610595703,19.830501556396484,-0.3216327726840973,2.6130337715148926,0.04033986549469937,0.01823640263202334,0.044388907661337296,0.0,0.1137638087262609,0.09485607097744764,0.5485841987547181,g1,LPV_SRV_AGB_O,0.39,0.0,647.9578075285723,LGGS J002022.90+591751.8,0.252989,Em*
5,168,zr,806203100060423,5.0989475,59.3023316,603,43.353397369384766,3.6096865088249213,9.805802856611858e-19,0.20166358351707458,5.211841335015464e-23,481.50270086755603,0.997336033066433,0.28793891233976,0.19382587404148513,19.980669021606445,0.20166358351707458,19.334941864013672,20.572986602783203,-0.2577386200428009,1.9497979879379272,0.02288762441783082,0.005203361869715385,0.015654582112170936,0.0,0.030238490340185997,0.06319318525503015,0.31436707892533344,g2,LPV_OSARG_AGB,0.26,0.0,503.96718363333395,[HL90] 92,1.336543,HII
6,210,zr,806203100060348,5.0498178,59.314207,524,46.50968933105469,7.160523506839386,1.1875559898120985e-83,0.2575588822364807,6.041918640486786e-47,2042.1167531644894,0.997343815080248,0.5563482426780246,0.48559856224556536,19.789920806884766,0.2575588822364807,19.26156234741211,20.64164161682129,-0.4833795726299286,2.7955565452575684,0.0395216855981239,0.06772269663685082,0.06811441803218907,0.0,0.9925813043509061,0.4743435395361756,0.755170773369338,g1,LPV_SRV_AGB_O,0.26,0.0,1508.9202546000013,[WM9

In [7]:
obj_zg[np.isnan(obj_zg['dist'])==0].show_in_notebook()

idx,SourceID,filter,OID,RA,DEC,numobs,DELTA_SIG_MAG,CONST_CHI2,sf,rmse,LNP_SIG,period,period_c,power,power_c,mean,std,min,max,sigma_mean,sigma_std,gaussian1_err,gaussian2_err,gumbel_err,g2_flag,Pg1g2,Pgug2,Pg1gu,shape,upsilon_class,upsilon_prob,upsilon_flag,upsilon_period,identifier,dist,type
0,73,zg,806103100048650,5.0758587,59.2958026,97,64.1142578125,8.935053528431245,1.364237212874404e-43,0.23146717250347137,1.5286033678611772e-06,1.0323211002809742,0.9972896868175051,0.35458712707540346,0.17918306600135617,20.327228546142578,0.23146717250347137,19.153165817260742,20.794662475585938,-0.3369257152080536,3.0768370628356934,0.009533430482035768,0.01393656832479377,0.05977027922886222,0.0,0.8494752978458127,0.031768959029891186,0.986429830676151,g1,CEPH_Other,0.49,1.0,2.150138196410262,[HL90] 46,1.931646,HII
1,77,zg,806103100029923,5.0869784,59.2997881,258,80.47639465332031,16.358047581022113,8.308402166762659e-291,0.31344184279441833,2.3170100655180286e-16,26.55360081018743,1.0000143393856216,0.29023605619128945,0.005352919590317105,20.02450942993164,0.31344184279441833,18.832887649536133,20.759401321411133,-0.705572783946991,4.285964488983154,0.012970672357106496,0.004716468677232082,0.04254821537149007,1.0,0.07510737992757988,0.007362311539822919,0.9301277856779838,g1,CEPH_Other,0.42,0.0,26.525643273684214,[WWW2005] XA 11,0.385353,X
2,90,zg,806103100049624,5.0834336,59.3114529,324,188.7042694091797,29.36702577363452,0.0,0.29480019211769104,3.044761670180407e-168,2043.2292424728973,1.0000143393856216,0.9125489407249586,0.43350509622734634,18.898109436035156,0.29480019211769104,18.297286987304688,19.611181259155273,-1.0131325721740723,5.650538921356201,0.23967239977166246,0.05805681475716322,0.1527728485996977,0.0,0.034221739839404575,0.08173142602727546,0.2834658939107344,g2,LPV_OSARG_AGB,0.31,0.0,1511.9616666,LGGS J002020.03+591841.2,0.158893,Em*


In [8]:
unobj_zr[np.isnan(unobj_zr['dist'])==0].show_in_notebook()

idx,SourceID,filter,OID,RA,DEC,numobs,DELTA_SIG_MAG,CONST_CHI2,sf,rmse,LNP_SIG,period,period_c,power,power_c,mean,std,min,max,sigma_mean,sigma_std,gaussian1_err,gaussian2_err,gumbel_err,g2_flag,Pg1g2,Pgug2,Pg1gu,shape,upsilon_class,upsilon_prob,upsilon_flag,upsilon_period,identifier,dist,type
0,34,zr,1809211300070171,5.0727284,59.3098249,31,81.30490112304688,7.451470886438102,2.3595134061374747e-11,0.1517023891210556,0.11556026631576935,0.1240045534413452,2800.0,0.43705000916041337,0.1948276490633615,19.289358139038086,0.1517023891210556,18.910890579223633,19.555213928222656,-0.28222447633743286,2.735348701477051,0.44073983462560923,0.5540522748460567,0.46764695860749705,1.0,0.6894512650780728,0.6300261912877945,0.5301494257802675,g1,DSCT,0.69,1.0,0.044120282725895896,NVSS J002017+591841,1.176509,Radio
1,34,zr,806203100060368,5.0725296,59.3098467,262,80.29171752929688,7.117226100332736,2.2607034617993476e-41,0.17434047162532806,0.008108579964863858,0.17204026613816514,0.997343815080248,0.15900530649983438,0.00012080295156057017,19.31197738647461,0.17434047162532806,18.88072967529297,19.74237823486328,-0.3230542242527008,2.718236207962036,0.019308752196863933,0.009141940373133643,0.07060550749620839,1.0,0.12415968294451318,0.010007429886732089,0.9456572622860415,g1,DSCT,0.88,1.0,0.17204168302675601,NVSS J002017+591841,1.176509,Radio
2,55,zr,806203100043559,5.0723453,59.3111226,855,1322.654541015625,28.139266597643587,0.0,0.09289497882127762,1.0401704349953329e-06,2800.0,0.997336033066433,0.09657835576966833,0.8283801394042668,16.962669372558594,0.09289497882127762,16.404556274414062,17.23210906982422,-0.1622304767370224,5.306269645690918,0.009358681227241613,0.001534619994822164,0.12127946984456363,1.0,0.015938986578472036,9.779189688419176e-05,0.9984113822662893,g1,NonVar,0.79,1.0,2267.852326350003,[BGR2000] Br 1,0.17367,HII
3,55,zr,1809211300066648,5.0724769,59.3111324,77,649.3062133789062,17.884333922325375,9.82310740598561e-98,0.06559567898511887,0.00021590831277816652,0.10674401734567869,2800.0,0.34204672407082565,0.021634072466851725,16.935136795043945,0.06559567898511887,16.740747451782227,17.062875747680664,-0.09860587120056152,4.204975605010986,2.648841206633338,0.4319534830074969,2.0249945408637027,1.0,0.015766331876517103,0.026698985916089324,0.3660444451778835,gu,DSCT,0.45,1.0,0.0964498720601044,[BGR2000] Br 1,0.17367,HII
4,64,zr,806203100060465,5.0739788,59.2961369,791,161.1138458251953,3.2559349557720485,1.0582204793748586e-14,0.07327186316251755,0.05186990648947234,0.10460713412988666,0.997336033066433,0.052003114693483725,0.014565343545433386,18.635385513305664,0.07327186316251755,18.284832000732422,18.8492374420166,-0.08440251648426056,1.8084462881088257,0.10513550464406803,0.008221874799986494,0.4474513471247722,1.0,0.0036880283230630306,0.00020599161789369713,0.9623955674043138,g1,NonVar,0.59,1.0,0.07596731740026137,[LL2015] 27,0.065818,Cl*
5,65,zr,806203100043833,5.0716716,59.2959753,843,324.9570007324219,3.601553304065856,6.578584336175542e-22,0.03792821615934372,1.8610094416490785e-08,2800.0,0.997336033066433,0.1164967305946571,0.8300275624372495,17.28755760192871,0.03792821615934372,17.128036499023438,17.391010284423828,-0.029083840548992157,1.8962376117706299,1.0915797822642355,0.06661462373404914,3.064348914344786,1.0,0.0022534176697209807,0.0002881254821144319,0.9016753593573754,g1,NonVar,0.89,0.0,4535.704652700006,[LL2015] 24,0.032157,Cl*
6,73,zr,806203100052194,5.0757586,59.2957439,570,79.02552795410156,4.492957767364811,2.0511315256049188e-32,0.14209401607513428,0.0013469039832887343,0.49968099722233067,0.9973357126290449,0.08833389447521292,0.056859390811042805,19.436952590942383,0.14209401607513428,18.890762329101562,19.732688903808594,-0.203139990568161,2.1489360332489014,0.03349849833893599,0.00722595301612069,0.13278539316689061,1.0,0.027289194603853928,0.0017941669502375968,0.9552115844818286,g1,RRL_ab,0.6,1.0,0.49969300037506753,[HL90] 46,1.871228,HII
7,73,zr,1809211300070260,5.0758365,59.295

In [9]:
unobj_zg[np.isnan(unobj_zg['dist'])==0].show_in_notebook()

idx,SourceID,filter,OID,RA,DEC,numobs,DELTA_SIG_MAG,CONST_CHI2,sf,rmse,LNP_SIG,period,period_c,power,power_c,mean,std,min,max,sigma_mean,sigma_std,gaussian1_err,gaussian2_err,gumbel_err,g2_flag,Pg1g2,Pgug2,Pg1gu,shape,upsilon_class,upsilon_prob,upsilon_flag,upsilon_period,identifier,dist,type
0,55,zg,806103100036162,5.0724243,59.3110593,337,524.6041259765625,11.53433716011322,3.585358205401398e-226,0.09109293669462204,6.624513205261614e-05,0.9972441069114794,1.0000143393856216,0.09943739652561774,0.904815666787983,17.759519577026367,0.09109293669462204,17.35228729248047,18.282495498657227,-0.11940788477659225,3.39772629737854,0.021082675533131933,0.0009674376856350767,0.058600720395744474,1.0,0.0012778740354683071,0.0001663393556028403,0.8995923437308072,g1,RRL_ab,0.53,1.0,0.4986132790810157,[BGR2000] Br 1,0.179192,HII
1,55,zg,1809111300028565,5.07245,59.3110868,66,392.635986328125,14.967355206033764,8.396331195386733e-66,0.09816054254770279,0.001972449838350469,0.22232818693454928,2800.0,0.33767960153347515,0.09093510017475841,17.740047454833984,0.09816054254770279,17.36347198486328,17.94055938720703,-0.16121967136859894,3.8505306243896484,0.379122196310376,0.1487498025445291,0.17974030374835914,1.0,0.0868797174594248,0.34586527658886057,0.1729739946754486,gu,DSCT,0.5,1.0,0.05968903932818899,[BGR2000] Br 1,0.179192,HII
2,79,zg,806103100049853,5.0841666,59.3103325,319,100.89814758300781,13.190935937807287,1.6236212003927169e-261,0.3389107882976532,0.9630887610262819,0.16739794432900304,1.0000143393856216,0.03901288339198156,0.007827767361578522,19.090709686279297,0.3389107882976532,18.44477653503418,20.729827880859375,-1.0078935623168945,3.8302736282348633,0.006344376912315118,0.0005656665842803147,0.006346515322447545,0.0,0.004783614293595195,0.004780419612124387,0.5001716322684765,g2,NonVar,0.52,1.0,0.06183049345419848,[TCA2017] T6,1.076874,WR*
3,85,zg,1809111300042593,5.0848266,59.3104646,63,141.77206420898438,9.276872126191114,9.143553620057548e-31,0.1382504105567932,0.07441019645986488,0.10729436912034605,2800.0,0.260849559494291,0.13702182318181275,18.98121452331543,0.1382504105567932,18.409698486328125,19.191980361938477,-0.23721453547477722,3.05719256401062,0.1080647020177061,0.051463802722698744,0.13516570580772344,1.0,0.12553472621604278,0.0820303747466492,0.6123250237883335,g1,DSCT,0.57,1.0,0.038396544192168085,LGGS J002020.35+591837.6,0.289957,Em*
4,85,zg,806103100049852,5.085006,59.3105231,329,224.17538452148438,7.025316375455554,6.21282558930419e-93,0.11896347999572754,0.0004236128690688058,0.9972441069114794,1.0000143393856216,0.09157741272952273,0.9078939327038209,19.00056266784668,0.11896347999572754,18.165077209472656,19.442018508911133,-0.16950754821300507,2.6658737659454346,0.008877441947716527,0.006429471672819983,0.010441253403287687,0.0,0.2726699506922211,0.20281337087079054,0.5820041377754086,g1,NonVar,0.49,1.0,0.0776094618838224,LGGS J002020.35+591837.6,0.289957,Em*
5,100,zg,806103100049623,5.0844871,59.311539,60,121.29397583007812,44.425590429699454,3.580400110681386e-239,0.49035248160362244,0.004218381083631,1.0001441927691062,1.0000110121384305,0.33303417110226397,0.9110818249847177,19.399503707885742,0.49035248160362244,18.297286987304688,20.46368408203125,-2.1311886310577393,7.429434776306152,0.06658749629043535,0.013527366068103966,0.07014360405237723,0.0,0.024267649787369616,0.021916109343879664,0.5264766425083548,g2,EB_EC,0.35,1.0,0.05015370325241265,[MAC92] 6,0.903892,WR*
6,119,zg,806103100042733,5.0876383,59.2962106,296,87.90541076660156,5.024397182504031,9.038635021729923e-41,0.15175144374370575,9.397199102478995e-05,0.25218106626800674,1.0000143393856216,0.10973730338686587,0.021996031938818078,19.505632400512695,0.15175144374370575,18.947853088378906,20.012239456176758,-0.21085329353809357,2.2690656185150146,0.013043866112065053,0.005979015925001082,0.06787238162481836,1.0,0.11678855631709106,0.00467065689444679,0.9775440558428248,g1,RRL_e,0.43,1.0,0.25217573802190474,LGGS J002021.06+59174